In [2]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 3.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 147 kB 72.9 MB/s 
  Created wheel for openai: filename=openai-0.25.0-py3-none-any.whl size=55880 sha256=a93067c69b4bdf478e0796568ddeb89241c6785c6d655c046c5256f7174a3c2e
  Stored in directory: /root/.cache/pip/wheels/4b/92/33/6f57c7aae0b16875267999a50570e81f15eecec577ebe05a2e
Successfully built openai


## Generating example CAA sentences using GPT-3

### Load files

In [1]:
!pip install gdown --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
!gdown https://drive.google.com/uc?id=1dC5iYiVwBOo5QxQBRtdC55Zzd6Qz6cVG

In [ ]:
import pandas as pd

SHUFFLED_SENTENCES = "./shuffled_sentences.txt"



### Open AI

In [ ]:
import openai, re

openai.api_key = "..."

def generate(prompt):
  response = openai.Completion.create(
    engine="davinci",
    prompt=prompt,
    temperature=0.7,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response['choices'][0]['text'];

### Generation


In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
GENERATED_EXAMPLES_PATH = "./new_sentences.txt"
new_sentence_file = open(GENERATED_EXAMPLES_PATH,"w")

In [ ]:
all_sentences = [l.rstrip() for l in open(SHUFFLED_SENTENCES,'r').readlines()]
len(all_sentences)

2602

In [ ]:
import random
prompts = []
for j in range(20):
  sentences = random.sample(all_sentences,k=10)
  prompt = "This is a list of distinct Portuguese sentences in direct order:\n\n"
  for i,sentence in enumerate(sentences):
    prompt += "Example {0}: {1} [END]\n\n".format(i+1,sentence)
  prompt += "Example {0}:".format(i+2)
  # print(prompt)
  res = generate(prompt)
  splited_sentences = re.split(r"\n\n", res) 
  sentences = ["".join(splited_sentences[0].split("[END]")).strip()]

  for sen in splited_sentences[1:]:
    finds = re.findall("(?<=:)(.*)(?=\[END\])",sen)
    if len(finds) > 0:
      sentences.append(finds[0].strip())
  prompts.append({
      "prompt": prompt,
      "sentences": sentences
  })
  for sentence in sentences:
    new_sentence_file.write(sentence+"\n")

new_sentence_file.close()

In [ ]:
from tqdm import tqdm
prompts = []
for sentences in tqdm(chunks(all_sentences,n=10)):
  prompt = "This is a list of distinct Portuguese sentences in direct order:\n\n"
  for i,sentence in enumerate(sentences):
    prompt += "Example {0}: {1} [END]\n\n".format(i+1,sentence)
  prompt += "Example {0}:".format(i+2)
  res = generate(prompt)
  splited_sentences = re.split(r"\n\n", res) 
  sentences = ["".join(splited_sentences[0].split("[END]")).strip()]

  for sen in splited_sentences[1:]:
    finds = re.findall("(?<=:)(.*)(?=\[END\])",sen)
    if len(finds) > 0:
      sentences.append(finds[0].strip())
  prompts.append({
      "prompt": prompt,
      "sentences": sentences
  })
  for sentence in sentences:
    new_sentence_file.write(sentence+"\n")

new_sentence_file.close()

67it [41:09, 36.86s/it]
